In [1]:
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
import tensorflow as tf 
from tensorflow.keras.models import load_model

In [3]:
modeltest1 = load_model("Aug_TPU_Sign_ES_P3.h5")
modeltest2 = load_model("Aug_TPU_Sign.h5")
modeltest3 = load_model("best_model_TPU.h5")

In [4]:
modeltest1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 128)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)             

In [5]:
modeltest2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 128)       1280      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)               

In [6]:
modeltest3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 128)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)             

In [7]:
import matplotlib.pyplot as plt

In [8]:
import pandas as pd
test_data = pd.read_csv('sign_mnist_test.csv')

In [9]:
test_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [10]:
test_labels = test_data['label']

In [11]:
test_data.drop('label', axis = 1, inplace = True)

In [12]:
test_data.shape

(7172, 784)

In [13]:
tests = test_data.values
test_imgs = np.array([np.reshape(i, (28,28)) for i in tests])
images = np.array([i.flatten() for i in test_imgs])
images.shape

(7172, 784)

Images taken 

In [14]:
cover = "abcdefghijklmnopqrstuvwxyz"

In [15]:
test_labels

0        6
1        5
2       10
3        0
4        3
        ..
7167     1
7168    12
7169     2
7170     4
7171     2
Name: label, Length: 7172, dtype: int64

In [16]:
import cv2

In [17]:
dri = r"D:\Study\Python_Scripts\SignLanguage MNIST"
cap = cv2.VideoCapture(0)#0 is internal cam : 1 is external webcam
i=0 #to save all the clicked images
while(True):
    ret, frame = cap.read()
    roi = frame[100:250, 100:250]
    cv2.rectangle(frame,(100,100),(250,250),(0,255,0),0)
    ycb = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    res = cv2.resize(ycb,(28,28))
    red=cv2.flip(res,1)
    np.array(res)
    imgs = res.reshape(1, 28, 28, 1)
    
    y_pred1 = modeltest1.predict(imgs)
    y_pred2 = modeltest2.predict(imgs)
    y_pre3 = modeltest3.predict(imgs)
    cv2.putText(frame, cover[np.argmax(y_pred1)], (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 1,True)
    cv2.putText(frame, cover[np.argmax(y_pred2)], (120, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1,True)
    cv2.putText(frame, cover[np.argmax(y_pre3)], (140, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 1,True)
    cv2.imshow("imshow",frame)
    
    key=cv2.waitKey(30)
    if key==ord('q'):
        break
    if key==ord('c'):
        i+=1
        cv2.imshow("imshow2",ycb)
        cv2.imwrite('tests\image' +str(i) +'.jpg', ycb)
        print("Wrote Image")

# release the capture
cap.release()
cv2.destroyAllWindows()